IGN

In [1]:
import pandas as pd
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

In [2]:
opciones = Options()
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.headless = False
opciones.add_argument('--start-maximized')
opciones.add_argument('--incognito')

In [3]:
PATH = 'drive/chromedriver'
driver = webdriver.Chrome(options=opciones)

In [4]:
url = "https://www.ign.es/web/ane-datos-geograficos/-/datos-geograficos/datosPoblacion?tipoBusqueda=capitales"
driver.get(url)

In [5]:
tabla = driver.find_element(By.XPATH, '//*[@id="p_p_id_IGNANEDatosGeograficos_WAR_IGNANEDatosGeograficosportlet_"]/div/div/div/div/table[2]')
tabla

<selenium.webdriver.remote.webelement.WebElement (session="a16af3991d396bfa08aea34f3ec5aebd", element="B83809F81CCE6122C138DF6D6E2CDD80_element_46")>

In [6]:
cuerpo = tabla.find_element(By.TAG_NAME, 'tbody')


filas = cuerpo.find_elements(By.TAG_NAME, 'tr')

In [7]:
datos_filas = []
for fila in filas:
    celdas = fila.find_elements(By.TAG_NAME, 'td')
    datos_fila = [celda.text for celda in celdas]
    datos_filas.append(datos_fila)

In [9]:
columnas = ['Capital', 'Provincia', 'Población', 'Latitud', 'Longitud', 'XUTM ETRS89', 'YUTM ETRS89', 
            'Altura (m)']  # Reemplaza con los nombres de tus columnas
data = pd.DataFrame(datos_filas, columns=columnas)

In [10]:
data.dtypes

Capital        object
Provincia      object
Población      object
Latitud        object
Longitud       object
XUTM ETRS89    object
YUTM ETRS89    object
Altura (m)     object
dtype: object

In [11]:
data= data.drop(columns=['XUTM ETRS89', 'YUTM ETRS89', 'Altura (m)'])

In [12]:
filas_eliminar= [0,13,30]
data= data.drop(filas_eliminar).reset_index(drop=True)
data

,Capital,Provincia,Población,Latitud,Longitud
0,A Coruña,A Coruña,246.047,"43,37012643","-8,39114853"
1,Albacete,Albacete,170.475,"38,99588053","-1,85574745"
2,Alicante / Alacant,Alicante / Alacant,334.418,"38,34548705","-0,4831832"
3,Almería,Almería,190.013,"36,83892362","-2,46413188"
4,Ávila,Ávila,58.245,"40,65586958","-4,69771277"
5,Badajoz,Badajoz,150.376,"38,87874339","-6,97099704"
6,Barcelona,Barcelona,1.619.337,"41,38424664","2,17634927"
7,Bilbao,Vizcaya,353.187,"43,25721957","-2,92390606"
8,Burgos,Burgos,178.574,"42,34113004","-3,70419805"
9,Cáceres,Cáceres,94.179,"39,47316762","-6,37121092"


In [13]:
correccion_provincias={'Álava': 'Araba/alava',
                      'Santa Cruz de Tenerife':'Sta. cruz de tenerife',
                      'Málaga':'Malaga',
                      'Las Palmas': 'Las palmas de gran canaria',
                      'Valencia / València':'Valencia',
                      'La Rioja': 'La rioja',
                      'León':'Leon',
                      'Alicante / Alacant':'Alicante',
                      'Almería':'Almeria',
                      'Ávila': 'Avila',
                      'Illes Balears': 'Islas baleares',
                      'Cáceres': 'Caceres',
                      'Cádiz':'Cadiz',
                      'Castellón / Castelló': 'Castellon',
                      'Ciudad Real': 'Ciudad real',
                      'Córdoba': 'Cordoba',
                      'A Coruña': 'A coruna',
                      'Jaén': 'Jaen' }

data['Provincia']= data['Provincia'].replace(correccion_provincias)

In [14]:
data = data.drop(columns=['Capital'])

In [15]:
data.dtypes

Provincia    object
Población    object
Latitud      object
Longitud     object
dtype: object

In [16]:
data['Población'] = pd.to_numeric(data['Población'].str.replace('.', ''), errors='coerce').astype(int)
data['Latitud'] = pd.to_numeric(data['Latitud'].str.replace(',', '.'), errors='coerce')
data['Longitud'] = pd.to_numeric(data['Longitud'].str.replace(',', '.'), errors='coerce')

In [17]:
data.dtypes

Provincia     object
Población      int32
Latitud      float64
Longitud     float64
dtype: object

In [18]:
data['ID_latitud_longitud']=range(len(data))

In [19]:
data.to_csv('../data/IGN.csv', index=False)